## Getting Started
Loading eye movement dataset with pymovements

In [1]:
import os
import pymovements as pm

# Ensure the data directory exists
os.makedirs('data/ToyDataset', exist_ok=True)

dataset = pm.Dataset(
    'ToyDataset',                # choose a public dataset from our dataset library
    path='data/ToyDataset',      # set up your local dataset path
)
dataset.download()               # download a public dataset from our dataset library
dataset.load()                   # download the dataset

Using already downloaded and verified file: data\ToyDataset\downloads\pymovements-toy-dataset.zip
Extracting pymovements-toy-dataset.zip to data\ToyDataset\raw


  0%|          | 0/20 [00:00<?, ?it/s]

Calculate velocities on the fly

In [2]:
dataset.pix2deg()                # transform pixel coordinates to degrees of visual angle
dataset.pos2vel()                # transform positional data to velocity data

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

and extract events with different eye movements event extraction algorithms

In [3]:
dataset.detect('ivt')            # detect fixation using the I-VT algorithm
dataset.detect('microsaccades')  # detect saccades using the microsaccades algorithm

0it [00:00, ?it/s]

0it [00:00, ?it/s]

## Upload and Select Stimulus (Video or Image)
Upload the stimulus video or image file.

In [4]:
import shutil
import ipyfilechooser
import cv2
import numpy as np
from IPython.display import display

# Step 1: Create the file chooser widget
chooser = ipyfilechooser.FileChooser(os.getcwd())  # Opens in the current directory
chooser.filter_pattern = ['*.mp4', '*.jpg', '*.jpeg', '*.png']  # Show video & image files
display(chooser)

# Step 2: Function to move the selected file or use the generated video
def save_uploaded_video():
    selected_file = chooser.selected  # Get selected file path

    if selected_file is None:
        # If no video or image is uploaded, generate a white background image
        default_image = "white_background.png"
        if not os.path.exists(default_image):
            print(f"No stimulus uploaded. Generating default white image: {default_image}")
            white_background = np.ones((480, 640, 3), dtype=np.uint8) * 255  # White 640x480 image
            cv2.imwrite(default_image, white_background)

        # Check if the generated image exists
        if os.path.exists(default_image):
            print(f"Using generated white image: {default_image}")
            selected_file = default_image
        else:
            print("ERROR: No stimulus uploaded and failed to generate an image. Please upload an image or video.")
            return

    # Move the selected file to 'stimulus.ext' (handle both video and image)
    stimulus_ext = os.path.splitext(selected_file)[-1].lower()
    stimulus_name = "stimulus" + stimulus_ext
    shutil.copy(selected_file, stimulus_name)

    # Verify the file exists
    if os.path.exists(stimulus_name):
        print(f"Stimulus saved successfully as '{stimulus_name}'!")
        print(f"File Size: {os.path.getsize(stimulus_name)} bytes")
    else:
        print("ERROR: Stimulus file was not saved correctly!")


FileChooser(path='C:\Users\Kirthan\IdeaProjects\pymovements-videoreplay', filename='', title='', show_hidden=F…

In [5]:
# Step 3: After Selecting the File, Run:
save_uploaded_video()

No stimulus uploaded. Generating default white image: white_background.png
Using generated white image: white_background.png
Stimulus saved successfully as 'stimulus.png'!
File Size: 2299 bytes


## Initialize and Run VideoPlayer
Run the VideoPlayer to visualize eye-tracking data overlay.

In [14]:
import sys
import importlib

# Ensure 'src' is in sys.path
src_path = os.path.abspath(os.path.join('src'))
if src_path not in sys.path:
    sys.path.append(src_path)

# Import the module
import videoreplay.video_player

# Reload the module (corrected)
importlib.reload(videoreplay.video_player)

# Now import the updated class
from videoreplay.video_player import VideoPlayer

# Determine stimulus file (image or video)
stimulus_files = [f for f in os.listdir() if f.startswith("stimulus.")]
if stimulus_files:
    stimulus_path = stimulus_files[0]
else:
    print("ERROR: No stimulus file found!")

# Initialize the VideoPlayer with the uploaded video
player = VideoPlayer(stimulus_path=stimulus_path,
                     dataset_path="data/ToyDataset",
                     dataset_name="ToyDataset")

  0%|          | 0/20 [00:00<?, ?it/s]

Checking video path: C:\Users\Kirthan\IdeaProjects\pymovements-videoreplay\stimulus.mp4
Video loaded successfully! FPS: 30.0, Total Frames: 516
Available columns in gaze_df: ['time', 'stimuli_x', 'stimuli_y', 'text_id', 'page_id', 'pixel']
frame_idx added! (Now properly scaled)
      time  normalized_time  frame_idx
0  1988145            0.000          0
1  1988146            0.001          0
2  1988147            0.002          0
3  1988148            0.003          0
4  1988149            0.004          0
🕒 Min Frame Index: 0
🕒 Max Frame Index: 515


## Play Video with Gaze Overlay

In [15]:
player.play(speed=1.0)  # Play at normal speed

Available columns in gaze_df: ['time', 'stimuli_x', 'stimuli_y', 'text_id', 'page_id', 'pixel', 'normalized_time', 'frame_idx']


## Navigate Through Fixations
Use 'n' for next fixation, 'p' for previous, 'q' to quit.

In [17]:
player.fixation_navigation()

## Export Gaze Replay as MP4
Save the replay with gaze overlay.

In [20]:
from IPython.display import Video

player.export_replay("gaze_replay.mp4")

# Display the exported video
Video("gaze_replay.mp4", embed=True)

KeyError: 'x'